<a href="https://colab.research.google.com/github/sbanwaskar/CS-479-579-Deep-Learning-Project/blob/main/Classifications_diff_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import networkx as nx
import pickle
import numpy as np
import pandas as pd
import scipy.stats as st
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


import numpy as np
from sklearn.impute import SimpleImputer 
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [ ]:
# #data import
# from google.colab import files
# uploaded = files.upload()
!wget https://www.dropbox.com/s/xkozscpcryu871h/labels_1540_4classes_icmla_21.pck
!wget https://www.dropbox.com/s/ovt8g99wjcz1mvr/mvts_1540_icmla_21.pck


In [ ]:
#reading pickle files
def load(file_name):
    with open(file_name, 'rb') as fp:
        obj = pickle.load(fp)
    return obj


Sampled_inputs=load("mvts_1540_icmla_21.pck")


Sampled_labels=load("labels_1540_4classes_icmla_21.pck")    

temp=Sampled_inputs[0]
print(temp)
df = pd.DataFrame(temp)
trainData=Sampled_inputs
trainLabel=Sampled_labels
print("trainData.shape: ",trainData.shape)
print("trainLebel.shape: ",trainLabel.shape)

In [ ]:
#standardization/z normalization of the univaraite time series
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
npArrays=[]
for l in range(0, len(trainData)):
  trainData_std = sc.fit_transform(trainData[l])
  #trainData_std = trainData_std.astype(np.float64)
  #print(type(trainData_std[0][0]))
  npArrays.append(trainData_std)

print(type(npArrays))
arr = np.asarray(npArrays)
print(type(arr))
trainData=arr
print("trainData.shape: ",trainData.shape)
print(type(trainData))
print("trainLebel.shape: ",trainLabel.shape)
print(type(trainLabel))

Saving all data to csv (different files)

In [ ]:
def time_series(trainData):

  rowList = []
#Here we parse through the entire trainData matrix and append eveything to the row list.
  for i in range(0, len(trainData)):

    for j in range(0, len(trainData[i])):
      
      rowList.append(trainData[i][j])
  # print(len(rowList))
  return rowList

def time_series1(rowList):
  rows, cols = (len(rowList), len(rowList[0])-1)

# here we declare a row list to which we append the values for all the 8 parameters.
  rowList1 = [[0 for i in range(cols)] for j in range(rows)]
#Here we parse through the entire trainData matrix and append eveything to the row list.
  for i in range (0, len(rowList)):

    for j in range(0,len(rowList[i])-1):

      rowList1[i][j] = rowList[i][j+1] - rowList[i][j]
  return rowList1

def get_mean_std(rowList):
  Mean = np.array(rowList).mean(1)
  means = np.reshape(Mean, (1540, 33))
  standardDeviation = np.array(rowList1).std(1)
  stdDev = np.reshape(standardDeviation, (1540, 33))
  return means,stdDev

def get_skew_and_kurtosis(rowList):
  df = pd.DataFrame(rowList)
  skew_df = df.skew(1)
  kurtosis_df = df.kurtosis(1)
  skew_rs = np.reshape(skew_df.values,(1540, 33))
  kurtosis_rs = np.reshape(kurtosis_df.values,(1540, 33))
  return skew_rs , kurtosis_rs

def get_all_params(rowList):
  mean , std_dev =get_mean_std(rowList)
  skew , kurtosis = get_skew_and_kurtosis(rowList)
  return mean,std_dev, skew,kurtosis

rowList = time_series(trainData)
rowList1 = time_series1(rowList)

mean,std_dev,skew , kurtosis = get_all_params(rowList)
mean1,std_dev1,skew1 , kurtosis1 = get_all_params(rowList1)

In [ ]:
all_params = [mean, std_dev,skew , kurtosis,mean1,std_dev1,skew1 , kurtosis1]
all_param_names = ['mean','std_dev','skew','kurtosis','mean1','std_dev1','skew1','kurtosis1']


## Method 1 : All Data in one csv please follow all_param_names to check the sequencing of data
# write
all_arrays_list = [a for a in all_params]
all_arrays_np = np.array(all_arrays_list).reshape(1540*8 , 33)
np.savetxt("all_params_data.csv", all_arrays_np, delimiter=",")
# read
data_read_back = np.loadtxt('all_params_data.csv',delimiter=',')
all_param_names = ['mean','std_dev','skew','kurtosis','mean1','std_dev1','skew1','kurtosis1']
all_param_data = data_read_back.reshape(8, 1540 , 33)


## Method 2 : All in one excel with different sheets (more organized, better when checking for issues in data by reading the excel manually)
# write-only
df_list = [pd.DataFrame(val) for val in all_params] 
filename = 'all_params_excel'
writer = pd.ExcelWriter(filename+'.xlsx')
for i, df in enumerate(df_list):
    df.to_excel(writer,sheet_name=all_param_names[i])
writer.save() 

In [ ]:
# All in one compressed pickle

final_data = {'mean':mean,
              'std_dev':std_dev,
              'skew':skew,
              'kurtosis':kurtosis,
              'mean1':mean1,
              'std_dev1':std_dev1,
              'skew1':skew1,
              'kurtosis1':kurtosis1}

import bz2
import pickle
import _pickle as cPickle

filename = 'all_data_in_one'

def decompress_pickle(file):
  data = bz2.BZ2File(file, 'rb')
  data = cPickle.load(data)
  return data

def compressed_pickle(title, data):
  with bz2.BZ2File(title + '.pbz2', 'w') as f: 
   cPickle.dump(data, f)

compressed_pickle(filename, final_data) 
data = decompress_pickle(filename+'.pbz2') 

In [ ]:
!zip -r all_data_in_one.zip all_*
from google.colab import files
files.download('all_data_in_one.zip') 

In [ ]:
def time_series():

  rowList = []
#Here we parse through the entire trainData matrix and append eveything to the row list.
  for i in range(0, len(trainData)):

    for j in range(0, len(trainData[i])):
      
      rowList.append(trainData[i][j])
  # print(len(rowList))
  return rowList

def time_series1(rowList):
  rows, cols = (len(rowList), len(rowList[0])-1)

# here we declare a row list to which we append the values for all the 8 parameters.
  rowList1 = [[0 for i in range(cols)] for j in range(rows)]
#Here we parse through the entire trainData matrix and append eveything to the row list.
  for i in range (0, len(rowList)):

    for j in range(0,len(rowList[i])-1):

      rowList1[i][j] = rowList[i][j+1] - rowList[i][j]
  return rowList1

In [ ]:
def mean():
  Mean = np.array(rowList).mean(1)
  means = np.reshape(Mean, (1540, 33))
  df = pd.DataFrame(means)
  print(means)
  df.to_csv("mean.csv")
def deviation():
  standardDeviation = np.array(rowList).std(1)
  stdDev = np.reshape(standardDeviation, (1540, 33))
  DF = pd.DataFrame(stdDev)
  print("info for deviation")
  print(DF.info)
  DF.to_csv("standardDeviation.csv")
def  skew():
  df = pd.DataFrame(rowList)
  skewValue = np.array(df.skew(1))
  skews = np.reshape(skewValue, (1540, 33))
  df = pd.DataFrame(skews)
  print("info for skew")
  print(df.info)
  df.to_csv("skew.csv")
def kurt():
  df = pd.DataFrame(rowList)
  kurtosisValue = np.array(df.kurt(1))
  kurts = np.reshape(kurtosisValue, (1540, 33))
  df = pd.DataFrame(kurts)
  print("info for kurtsosis")
  print(df.info)
  df.to_csv("kurt.csv")



In [ ]:
def mean():
  Mean = np.array(rowList).mean(1)
  means = np.reshape(Mean, (1540, 33))
  df = pd.DataFrame(means)
  print(means)
  df.to_csv("mean.csv")
def deviation():
  standardDeviation = np.array(rowList).std(1)
  stdDev = np.reshape(standardDeviation, (1540, 33))
  DF = pd.DataFrame(stdDev)
  print("info for deviation")
  print(DF.info)
  DF.to_csv("standardDeviation.csv")
def  skew():
  df = pd.DataFrame(rowList)
  skewValue = np.array(df.skew(1))
  skews = np.reshape(skewValue, (1540, 33))
  df = pd.DataFrame(skews)
  print("info for skew")
  print(df.info)
  df.to_csv("skew.csv")
def kurt():
  df = pd.DataFrame(rowList)
  kurtosisValue = np.array(df.kurt(1))
  kurts = np.reshape(kurtosisValue, (1540, 33))
  df = pd.DataFrame(kurts)
  print("info for kurtsosis")
  print(df.info)
  df.to_csv("kurt.csv")



In [ ]:
def time_series1(rowList):
  rows, cols = (len(rowList), len(rowList[0])-1)

# here we declare a row list to which we append the values for all the 8 parameters.
  rowList1 = [[0 for i in range(cols)] for j in range(rows)]
#Here we parse through the entire trainData matrix and append eveything to the row list.
  for i in range (0, len(rowList)):

    for j in range(0,len(rowList[i])-1):

      rowList1[i][j] = rowList[i][j+1] - rowList[i][j]

  return rowList1
    #print(len(rowList))
# the size of the row list is 1540*33 = 50820.


      






  


In [ ]:
Mean = np.array(rowList1).mean(1)
means = np.reshape(Mean, (1540, 33))
df = pd.DataFrame(means)
df.to_csv("mean.csv")

In [ ]:
def mean1():
  Mean = np.array(rowList1).mean(1)
  means = np.reshape(Mean, (1540, 33))
  df = pd.DataFrame(means)
  df.to_csv("mean1.csv")
def deviation1():
  standardDeviation = np.array(rowList1).std(1)
  stdDev = np.reshape(standardDeviation, (1540, 33))
  DF = pd.DataFrame(stdDev)
  DF.to_csv("standardDeviation1.csv")
def skew1():
  df = pd.DataFrame(rowList1)
  skewValue = np.array(df.skew(1))

#print(len(skewValue))
  skews = np.reshape(skewValue, (1540, 33))
  df = pd.DataFrame(skews)
  df.to_csv("skew1.csv")
def kurt1():
  df = pd.DataFrame(rowList1)
  kurtosisValue = np.array(df.kurt(1))
#print(len(skewVale))
  kurts = np.reshape(kurtosisValue, (1540, 33))
  df = pd.DataFrame(kurts)
  df.to_csv("kurt1.csv")
#size of kurtosis is 1540*33



In [ ]:
def main_mean():
 df = pd.read_csv("all_params_data.csv")

In [ ]:
main_mean()

In [ ]:
from scipy.stats import loguniform
from pandas import read_csv
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
#Here change the name of the file in [''] to check accuracy of required file
	# ['mean'] ['std_dev'] ['skew'] ['kurtosis'] ['mean1'] ['std_dev1'] ['skew1'] ['kurtosis']
X = decompress_pickle('all_data_in_one.pbz2')['kurtosis1']
Y = trainLabel
X.shape,Y.shape

((1540, 33), (1540,))

**KNN Classification**
**/SVM**





In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import RepeatedStratifiedKFold
from xgboost import XGBClassifier

In [ ]:
 from sklearn.model_selection import train_test_split
 X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.30)
 from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.30)


In [ ]:
#using polynomial kernel
from sklearn.svm import SVC
svclassifier_polynomail = SVC(kernel='poly', degree=8)
svclassifier_polynomail.fit(X_train, Y_train)
#Y_pred = svclassifier.predict(X_test)
Y_pred = svclassifier_polynomail.predict(X_test)
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(Y_test, Y_pred))
print(classification_report(Y_test, Y_pred))

In [ ]:
 #using gaussian rbf kernel
svclassifier_gauss = SVC(kernel='rbf',random_state=0)
svclassifier_gauss.fit(X_train, Y_train)
Y_pred = svclassifier_gauss.predict(X_test)
print(confusion_matrix(Y_test, Y_pred))
print(classification_report(Y_test, Y_pred))


In [ ]:
#using sigmoid kernel

svclassifier_sigmoid = SVC(kernel='sigmoid',random_state=0)
svclassifier_sigmoid.fit(X_train, Y_train)
Y_pred = svclassifier_sigmoid.predict(X_test)
print(confusion_matrix(Y_test, Y_pred))
print(classification_report(Y_test, Y_pred))


In [ ]:
#using linear kernel
svclassifier_linear = SVC(kernel='linear')
svclassifier_linear.fit(X_train, Y_train)
Y_pred = svclassifier_linear.predict(X_test)
print(confusion_matrix(Y_test, Y_pred))
print(classification_report(Y_test, Y_pred))


In [ ]:

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
#from sklearn.cross_validation import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

# Function importing Dataset

# Function to split the dataset
def splitdataset(balance_data):

	# Separating the target variable
	#Here change the name of the file in [''] to check accuracy of required file
	# ['mean'] ['std_dev'] ['skew'] ['kurtosis'] ['mean1'] ['std_dev1'] ['skew1'] ['kurtosis']
	X = decompress_pickle('all_data_in_one.pbz2')['std_dev1']
	Y = trainLabel

	# Splitting the dataset into train and test
	X_train, X_test, y_train, y_test = train_test_split(
	X, Y, test_size = 0.3, random_state = 100)
	
	return X, Y, X_train, X_test, y_train, y_test
	
# Function to perform training with giniIndex.
def train_using_gini(X_train, X_test, y_train):

	# Creating the classifier object
	clf_gini = DecisionTreeClassifier(criterion = "gini",
			random_state = 100,max_depth=3, min_samples_leaf=5)

	# Performing training
	clf_gini.fit(X_train, y_train)
	return clf_gini
	
# Function to perform training with entropy.
def tarin_using_entropy(X_train, X_test, y_train):

	# Decision tree with entropy
	clf_entropy = DecisionTreeClassifier(
			criterion = "entropy", random_state = 100,
			max_depth = 3, min_samples_leaf = 5)

	# Performing training
	clf_entropy.fit(X_train, y_train)
	return clf_entropy


# Function to make predictions
def prediction(X_test, clf_object):

	# Predicton on test with giniIndex
	y_pred = clf_object.predict(X_test)
	print("Predicted values:")
	print(y_pred)
	return y_pred
	
# Function to calculate accuracy
def cal_accuracy(y_test, y_pred):
	
	print("Confusion Matrix: ",
		confusion_matrix(y_test, y_pred))
	
	print ("Accuracy : ",
	accuracy_score(y_test,y_pred)*100)
	
	print("Report : ",
	classification_report(y_test, y_pred))


def main():
	
  #Here change the name of the file in [''] to check accuracy of required file
	# ['mean'] ['std_dev'] ['skew'] ['kurtosis'] ['mean1'] ['std_dev1'] ['skew1'] ['kurtosis']
	data = decompress_pickle('all_data_in_one.pbz2')['std_dev1']
	X, Y, X_train, X_test, y_train, y_test = splitdataset(data)
	clf_gini = train_using_gini(X_train, X_test, y_train)
	clf_entropy = tarin_using_entropy(X_train, X_test, y_train)
	
	# Operational Phase
	print("Results Using Gini Index:")
	
	# Prediction using gini
	y_pred_gini = prediction(X_test, clf_gini)
	cal_accuracy(y_test, y_pred_gini)
	
	print("Results Using Entropy:")
	# Prediction using entropy
	y_pred_entropy = prediction(X_test, clf_entropy)
	cal_accuracy(y_test, y_pred_entropy)
	
	
# Calling main function
if __name__=="__main__":
	main()


##KNN Classification


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test =  train_test_split(X, Y, test_size = 0.30,random_state = 17)
#X_train, Y_test = train_test_split(df, test_size = 0.20, random_state= 17)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.fit_transform(X_test)
# X_train =  scaler.transform(X_train)
# X_test  =  scaler.transform(X_test)

In [ ]:
from sklearn import preprocessing
from sklearn import utils

lab_enc = preprocessing.LabelEncoder()
encoded = lab_enc.fit_transform(Y_train)


print(utils.multiclass.type_of_target(Y_train))

print(utils.multiclass.type_of_target(Y_train.astype('int')))


print(utils.multiclass.type_of_target(encoded))


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report,accuracy_score
import seaborn as sns
def get_knn(X_train,Y_train,X_test,Y_test,n_neighbors):
  neigh = KNeighborsClassifier(n_neighbors=n_neighbors)
  neigh.fit(X_train, Y_train)
  accuracy = accuracy_score(neigh.predict(X_test),Y_test)
  print('Accuracy:',accuracy,'\n\n')
  print(classification_report(neigh.predict(X_test),Y_test))
  return accuracy

In [ ]:
accuracy_list = []
n_neighbor_list = []
accuracy_std_list = []
for i in range(1,11):
  print("KNN for n= ",i," after flattening - (1540 * 8 x 33) :\n")
  accuracy = get_knn(X_train,Y_train,X_test,Y_test,n_neighbors=i)
  print("KNN after standardization  for n= ",i,":\n")
  accuracy_std = get_knn(X_train_std,Y_train,X_test_std,Y_test,n_neighbors=i)
  accuracy_list.append(accuracy)
  n_neighbor_list.append(i)
  accuracy_std_list.append(accuracy_std)

## LSTM + 1D Convolution

In [151]:
# #@title imports
# import numpy as np
# import pandas as pd
# import networkx as nx
# import pickle

# import json
# import pandas as pd
# import numpy as np
# from numpy import array
# from numpy import hstack
# from keras.models import Sequential
# #from keras.layers import LSTM
# #from keras.layers import Dense
# #import scipy.stats as st

# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import torch.optim as optim
# from sklearn.model_selection import train_test_split

# from datetime import datetime
# from sklearn import preprocessing
# from sklearn import metrics
# from sklearn.metrics import classification_report
# from sklearn.metrics import accuracy_score, confusion_matrix, adjusted_rand_score

# import torch
# import torch.nn as nn
# #import matplotlib.pyplot as plt
# import numpy as np
# #import matplotlib.pyplot as plt
# import pandas as pd
# import torch
# import torch.nn as nn
# from torch.autograd import Variable
# from sklearn.preprocessing import MinMaxScaler
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import torch.optim as optim

# torch.manual_seed(1)
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# print("device: ", device)

device:  cpu


In [ ]:
#data import
# # from google.colab import files
# # uploaded = files.upload()
# !wget https://www.dropbox.com/s/xkozscpcryu871h/labels_1540_4classes_icmla_21.pck
# !wget https://www.dropbox.com/s/ovt8g99wjcz1mvr/mvts_1540_icmla_21.pck






In [ ]:
 #reading pickle files
# def load(file_name):
#     with open(file_name, 'rb') as fp:
#         obj = pickle.load(fp)
#     return obj


# Sampled_inputs=load("mvts_1540_icmla_21.pck")

# Sampled_labels=load("labels_1540_4classes_icmla_21.pck")    

# # temp=Sampled_inputs[0]
# # print(temp)
# # df = pd.DataFrame(temp)
# trainData=Sampled_inputs
# trainLabel=Sampled_labels
# trainLebel=Sampled_labels
# print("trainData.shape: ",trainData.shape)
# print("trainLebel.shape: ",trainLabel.shape)

In [ ]:
# from sklearn.preprocessing import StandardScaler
# sc = StandardScaler()
# trainData_std = sc.fit_transform(trainData.reshape(trainData.shape[0]*trainData.shape[1],trainData.shape[2]))
# trainData_std = trainData_std.reshape(trainData.shape)

# # Normal Input
# input_data = torch.from_numpy(trainData).float()

# # # Standardized Input
# # input_data = torch.from_numpy(trainData_std).float()

In [ ]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import torch.optim as optim

# torch.manual_seed(1)

# lstm = nn.LSTM(60, 60)  # Input dim is 3, output dim is 3
# # inputs = [torch.randn(1, 3) for _ in range(5)]  # make a sequence of length 5

# inputs = input_data
# hidden = (torch.randn(1, 33, 60),
#           torch.randn(1, 33, 60))

# output, hidden = lstm(inputs, hidden)

# # for i in inputs:
# #     # Step through the sequence one element at a time.
# #     # after each step, hidden contains the hidden state.
# #     out, hidden = lstm(i.view(1, 33, -1), hidden)

In [ ]:
# #standardization/z normalization of the univaraite time series
# from sklearn.preprocessing import StandardScaler
# sc = StandardScaler()
# npArrays=[]
# for l in range(0, len(trainData)):
#   trainData_std = sc.fit_transform(trainData[l])
#   #trainData_std = trainData_std.astype(np.float64)
#   #print(type(trainData_std[0][0]))
#   npArrays.append(trainData_std)

# print(type(npArrays))
# arr = np.asarray(npArrays)
# print(type(arr))
# trainData=arr
# print("trainData.shape: ",trainData.shape)
# print(type(trainData))
# print("trainLebel.shape: ",trainLabel.shape)
# print(type(trainLabel))

##Model (Load Data, Train , Test)

In [ ]:
# new_train_data = output.detach().clone().numpy()

# temptrainData=np.empty([1540,60, 33])
# n=len(new_train_data)
# for l in range(0, n):
#   temp=new_train_data[l]
#   #print(temp)
#   #temp=np.transpose(temp)
#   temp=temp.T
#   # print(temp.shape)
#   #print(temp)
#   temptrainData[l,:,:]=temp
#   n=n+1
#   #np.append(temptrainData, temp)
#   #print(temptrainData)

# #print(temptrainData.shape)
# #print(trainData.shape) 
# trainData=temptrainData
# print("trainData.shape: ",trainData.shape)
# #print(trainData[0])

# temp=new_train_data[0]
# #print(temp)
# df = pd.DataFrame(temp)
# #df=pd.DataFrame.from_dict(trainData)
# trainData222=new_train_data

In [ ]:
# INPUT_DIM = 33
# HIDDEN_DIM = 64
# NUM_TS = 60
# NUM_CLASSES = 4
# num_layers = 1 #number of stacked lstm layers
# hidden_size=HIDDEN_DIM

In [ ]:
# class bcolors:
#     HEADER = '\033[95m'
#     OKBLUE = '\033[94m'
#     OKCYAN = '\033[96m'
#     OKGREEN = '\033[92m'
#     WARNING = '\033[93m'
#     FAIL = '\033[91m'
#     ENDC = '\033[0m'
#     BOLD = '\033[1m'
#     UNDERLINE = '\033[4m'

#  Model with 1D Convolution


In [ ]:

# # input_dim = 33, hidden_dim = 128, num_TS = 60, num_classes = 4

# from collections import OrderedDict
# class LSTM_MVTS_LRN(nn.Module):
#   def __init__(self, input_dim, hidden_dim, num_classes):
#     super(LSTM_MVTS_LRN, self).__init__()
#     self.hidden_dim = hidden_dim
  
#     # nn.Conv1D(input_shape=(input_dim,NUM_TS)
#     kernel_size = 2
#     # self.conv1  = nn.Conv1d(1,hidden_dim,kernel_size = kernel_size)
#     # self.conv2  = nn.Conv1d(hidden_dim,1,kernel_size = kernel_size)

#     # self.conv1  = nn.Conv1d(1,hidden_dim,kernel_size = kernel_size,stride=2)
#     # input_dim_afterConv1d = int(np.ceil(input_dim/2)) + 1
#     # self.conv2  = nn.Conv1d(input_dim_afterConv1d,hidden_size,kernel_size = kernel_size)

#     self.conv1  = nn.Conv1d(1,hidden_dim,kernel_size = kernel_size)
#     input_dim_afterConv1d = int(input_dim - kernel_size)
#     self.conv2  = nn.Conv1d(hidden_dim,1,kernel_size = kernel_size)
    
#     self.maxpool1d = nn.MaxPool1d(1)

   
    
#     # Initialize the LSTM
#     self.lstm = nn.LSTM(input_dim_afterConv1d, hidden_dim)
    
    
#     # Initialize the RNN
#     self.RNN = nn.RNN(input_dim_afterConv1d, hidden_dim)

#       # nn.Conv1D(input_shape=(input_dim,NUM_TS))
#       #(input_size=INPUT_SIZE, hidden_size=HIDDEN_SIZE, num_layers = 1, batch_first=True)


#     self.hidden2class = nn.Linear(hidden_dim, num_classes)
#   def forward(self, mvts):
#     #input single mvts (60, 33); output class probability vector (1,4)

#     mvts_3d = mvts.view(len(mvts), 1, -1) #  mvts.shape: (60, 33); len(mvts)=60; new shape (mvts_3d): (60, 1, 33); 

#     # 1d Convolution with ReLU activation 
#     # x = F.relu(self.conv1(mvts_3d))
#     # x = F.relu(self.conv2(x))

#     ## 1d Convolution with Maxpooling
#     x = self.conv1(mvts_3d)
#     x = self.maxpool1d(x)
#     x = self.conv2(x)
#     x = self.maxpool1d(x)
    

#     ## LSTM Layer

#     # lstm_out,_ = self.lstm(x)
#     # ##last_lstm_out = torch.cat((lstm_out[0],lstm_out[1]),dim=0).mean(0).unsqueeze(0) #(1,64)
#     # last_lstm_out = lstm_out[len(lstm_out)-1]
#     # final_output = last_lstm_out

#     ## RNN Layer

#     RNN_out,_ = self.RNN(x)
#     # last_RNN_out = torch.cat((RNN_out[0],RNN_out[1]),dim=0).mean(0).unsqueeze(0) #(1,64)
#     last_RNN_out = RNN_out[len(RNN_out)-1]
#     final_output = last_RNN_out
    
#     # print(final_output.shape)
#     class_space = self.hidden2class(final_output) #(1,4)
#     class_scores = F.log_softmax(class_space, dim=1)
#     return class_scores


### Dowork - Train

In [ ]:

# import random
# import matplotlib.pyplot as plt

# def doWork( X_train, X_test, y_train, y_test):

#     num_masterIteration=1

#     #print("X_train.shape: ",X_train.shape)
#     #print("X_test.shape: ",X_test.shape)
#     #print("y_train.shape: ",y_train.shape)
#     #print("y_test.shape: ",y_test.shape)
#     #print(type(X_train))

#     classification_report_dict=[]
#     Accuracy=[]
#     for masterIteration in range(num_masterIteration):
#         print("\nmasterIteration: ",masterIteration)
#         #print(bcolors.WARNING + "\nmasterIteration :" + bcolors.WARNING,masterIteration)
#         random_state=random.randint(42, 100)
#         #print("random_state: ",random_state)


#         model = LSTM_MVTS_LRN(INPUT_DIM, HIDDEN_DIM, NUM_CLASSES)
#         loss_function = nn.NLLLoss()
#         optimizer = optim.SGD(model.parameters(), lr=0.01)
#         numTrain = X_train.shape[0]

#         numEpochs =1

#         #train
#         for epoch in range(numEpochs):
#           print("\n nmasterIteration, epoch: ",masterIteration,epoch)
#           loss_values = []
#           running_loss = 0.0


#           for i in range(numTrain):
#             model.zero_grad()
#             mvts = X_train[i,:,:]
#             mvts = torch.from_numpy(mvts).float()


#             target = y_train[i]
#             #print(type(target))
#             target = [target]
#             #print(type(target))
#             target=np.array(target)
#             #print(type(target))
#             target = torch.Tensor(target)
#             #print(type(target))

#             target = target.type(torch.LongTensor)

#             #print(type(mvts))
#             #print(mvts.is_cuda)
#             mvts = mvts.to(device)
#             #print(mvts.is_cuda)
#             #print(type(target))
#             #print(target.is_cuda)
#             target = target.to(device)
#             #print(target.is_cuda)

#             #target = torch.from_numpy(np.array(target))
#             mvts = mvts.view(mvts.size(0), -1)
#             model.to(device)
   
#             class_scores = model(mvts)
           
#             # print(class_scores)
#             #print(target)

#             loss = loss_function(class_scores, target)
                     
#             #loss = criterion_label(outputs, labels)
#             #print(loss)
#             loss.backward()
#             optimizer.step()
#             running_loss += loss.item()

#             #loss_values.append(running_loss)
#             loss_values.append(running_loss / len(X_train))
            
#           maxAcc=0
#           max_classification_report_dict=0
#           #print("loss: ",loss)  
#           #test 
#           numTest = X_test.shape[0]
#           with torch.no_grad():
#               numCorrect = 0
#               testLabel=[]
#               predictaedLabel=[]
#               for i in range(numTest):
#                 test_mvts = X_test[i,:,:]
#                 test_label = y_test[i] #class = 2

#                 #model.zero_grad()

#                 test_mvts = torch.from_numpy(test_mvts).float()

#                 test_mvts = test_mvts.to(device)
#                 #test_label = test_label.to(device)

#                 #print("\n\n1.test_label: ", test_label)
#                 test_class_scores = model(test_mvts) #test mvts = [0.35, 0.15, 0.45, 0.05]
#                 #print("2.1 test_class_scores: ",test_class_scores)
#                 #print(" type(test_class_scores): ",type(test_class_scores)) 
#                 class_prediction = torch.argmax(test_class_scores, dim=-1) #2
#                 #print("2.2 class_prediction: ", class_prediction)

#                 current_seq = np.argmax(test_class_scores.cpu().numpy())
#                 #print("4 current_seq: ",current_seq)
#                 #print("4 np.argmax(current_seq): ",current_seq))

#                 #print("3.0 class_prediction == test_label:", class_prediction == test_label)
#                 testLabel.append(test_label)
#                 predictaedLabel.append(current_seq)



#                 if(class_prediction == test_label): #(2,3 ) match 
#                   numCorrect = numCorrect+1
#               acc = numCorrect/numTest
#               #print("acc:", acc) 

#               #print("testLabel[]: ",testLabel)
#               #print("predictaedLabel[]: ",predictaedLabel)

#               fgdg=round(acc, 2)
#               #print("fgdg:", fgdg) 
#               if fgdg  > maxAcc:
#                 maxAcc=acc
#                 print(bcolors.WARNING + "maxAcc:" + bcolors.ENDC,maxAcc)
#                 max_classification_report_dict=metrics.classification_report(testLabel, predictaedLabel, digits=3,output_dict=True)
#         print(np.array(loss_values))
#         plt.plot(np.array(loss_values), 'r')
#         plt.xlabel('Epochs')
#         plt.ylabel('Total_Loss')
#         classification_report_dict.append(max_classification_report_dict)   
#         #print('classification_report_dict : \n',classification_report_dict)
#         Accuracy.append(maxAcc)  
#         #print('Accuracy : \n',Accuracy)
#     mean_acc = doClassSpecificCalulcation(Accuracy,trainLebel,classification_report_dict)
#     return mean_acc


In [ ]:
# def doClassSpecificCalulcation(Accuracy,trainLebel,classification_report_dict):
#   mean_acc = np.mean(Accuracy)
#   print('\np.mean(Accuracy) :',np.mean(Accuracy))
#   print('\np.std(Accuracy) :',np.std(Accuracy))
#   print('\n33333333 p.mean np.std(Accuracy) :     ',np.round(np.mean(Accuracy),2),"+-",np.round(np.std(Accuracy),2) )
#   for j in range( len(np.unique(trainLebel)) ):
#     print('\n\n\n\nclass :',j) 
#     precision=[]
#     recall=[]
#     f1_score=[]
#     for i in range(len(classification_report_dict)):
#       report=classification_report_dict[i]
#       #print('classification_report : \n',report) 
#       temp=report[str(j)]['precision'] 
#       precision.append(temp)

#       temp=report[str(j)]['recall'] 
#       recall.append(temp)

#       temp=report[str(j)]['f1-score'] 
#       f1_score.append(temp)

#     print('\np.mean(precision) \t p.mean(recall) \t p.mean(f1_score) :') 


#     print(np.mean(precision)) 
#     print(np.mean(recall)) 
#     print(np.mean(f1_score))

#     print('\np.mean p.std(precision) \tp.mean  p.std(recall) \tp.mean  p.std(f1_score) :')

#     print(np.round(np.mean(precision),2),"+-",np.round(np.std(precision),2) )
#     print(np.round(np.mean(recall),2),"+-",np.round(np.std(recall),2) )
#     print(np.round(np.mean(f1_score),2),"+-",np.round(np.std(f1_score),2) )
#   return mean_acc

In [ ]:
# def doWork2():

#   #test_sizes=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
#   test_sizes=[0.3]
#   for temp4 in range(len(test_sizes)):
#     test_size=test_sizes[temp4]
#     print("\n\n\n *************** test_size: ",test_size)
#     random_state=random.randint(42, 100)
#     print("random_state: ",random_state)
    
#     X_train, X_test, y_train, y_test = train_test_split(trainData, trainLebel, test_size=test_size, random_state=random_state)
#     #print("X_train.shape: ",X_train.shape)
#     #print("X_test.shape: ",X_test.shape)
#     #print("y_train.shape: ",y_train.shape)
#     #print("y_test.shape: ",y_test.shape)
#     #print(type(X_train))



#     print("X_train.shape X_test.shape y_train.shape y_test.shape ",
#               X_train.shape, X_test.shape ,y_train.shape, y_test.shape)
#     mean_acc = doWork( X_train, X_test, y_train, y_test)
#   return mean_acc

## **Run Main (doWork2)**

In [ ]:
#doWork2()

# Original Model

In [ ]:
# class LSTM_MVTS_LRN(nn.Module):
#   def __init__(self, input_dim, hidden_dim, num_classes):
#     super(LSTM_MVTS_LRN, self).__init__()
#     self.hidden_dim = hidden_dim
      
#     #self.lstm = nn.LSTM(input_dim, hidden_dim)
#     # Initialize the RNN.
#     self.lstm = nn.RNN(input_dim, hidden_dim)
#       #(input_size=INPUT_SIZE, hidden_size=HIDDEN_SIZE, num_layers = 1, batch_first=True)

#     self.hidden2class = nn.Linear(hidden_dim, num_classes)
#   def forward(self, mvts):
#     #print(mvts.shape)
#     #input single mvts (60, 33); output class probability vector (1,4)
#     lstm_out, _ = self.lstm(mvts.view(len(mvts), 1, -1)) #mvts.shape: (60, 33); len(mvts)=60; new shape: (60, 1, 33); lstm_out --> (60, 128)
#     last_lstm_out = lstm_out[len(lstm_out)-1] #(1,128)
#     class_space = self.hidden2class(last_lstm_out) #(1,4)
#     class_scores = F.log_softmax(class_space, dim=1)
#     return class_scores

### Dowork - Train

In [ ]:

# import random
# import matplotlib.pyplot as plt

# def doWork( X_train, X_test, y_train, y_test):

#     num_masterIteration=1

#     #print("X_train.shape: ",X_train.shape)
#     #print("X_test.shape: ",X_test.shape)
#     #print("y_train.shape: ",y_train.shape)
#     #print("y_test.shape: ",y_test.shape)
#     #print(type(X_train))

#     classification_report_dict=[]
#     Accuracy=[]
#     for masterIteration in range(num_masterIteration):
#         print("\nmasterIteration: ",masterIteration)
#         #print(bcolors.WARNING + "\nmasterIteration :" + bcolors.WARNING,masterIteration)
#         random_state=random.randint(42, 100)
#         #print("random_state: ",random_state)


#         model = LSTM_MVTS_LRN(INPUT_DIM, HIDDEN_DIM, NUM_CLASSES)
#         loss_function = nn.NLLLoss()
#         optimizer = optim.SGD(model.parameters(), lr=0.01)
#         numTrain = X_train.shape[0]

#         numEpochs =1

#         #train
#         for epoch in range(numEpochs):
#           print("\n nmasterIteration, epoch: ",masterIteration,epoch)
#           loss_values = []
#           running_loss = 0.0


#           for i in range(numTrain):
#             model.zero_grad()
#             mvts = X_train[i,:,:]
#             mvts = torch.from_numpy(mvts).float()


#             target = y_train[i]
#             #print(type(target))
#             target = [target]
#             #print(type(target))
#             target=np.array(target)
#             #print(type(target))
#             target = torch.Tensor(target)
#             #print(type(target))

#             target = target.type(torch.LongTensor)

#             #print(type(mvts))
#             #print(mvts.is_cuda)
#             mvts = mvts.to(device)
#             #print(mvts.is_cuda)

#             #print(type(target))
#             #print(target.is_cuda)
#             target = target.to(device)
#             #print(target.is_cuda)

#             #target = torch.from_numpy(np.array(target))
#             mvts = mvts.view(mvts.size(0), -1)
#             model.to(device)
   
#             class_scores = model(mvts)
#             # print(class_scores)
#             #print(target)

#             loss = loss_function(class_scores, target)
#             #loss = criterion_label(outputs, labels)
#             #print(loss)
#             loss.backward()
#             optimizer.step()
#             running_loss += loss.item()

#             #loss_values.append(running_loss)
#             loss_values.append(running_loss / len(X_train))

#           maxAcc=0
#           max_classification_report_dict=0
#           #print("loss: ",loss)  
#           #test 
#           numTest = X_test.shape[0]
#           with torch.no_grad():
#               numCorrect = 0
#               testLabel=[]
#               predictaedLabel=[]
#               for i in range(numTest):
#                 test_mvts = X_test[i,:,:]
#                 test_label = y_test[i] #class = 2

#                 #model.zero_grad()

#                 test_mvts = torch.from_numpy(test_mvts).float()

#                 test_mvts = test_mvts.to(device)
#                 #test_label = test_label.to(device)

#                 #print("\n\n1.test_label: ", test_label)
#                 test_class_scores = model(test_mvts) #test mvts = [0.35, 0.15, 0.45, 0.05]
#                 #print("2.1 test_class_scores: ",test_class_scores)
#                 #print(" type(test_class_scores): ",type(test_class_scores)) 
#                 class_prediction = torch.argmax(test_class_scores, dim=-1) #2
#                 #print("2.2 class_prediction: ", class_prediction)

#                 current_seq = np.argmax(test_class_scores.cpu().numpy())
#                 #print("4 current_seq: ",current_seq)
#                 #print("4 np.argmax(current_seq): ",current_seq))

#                 #print("3.0 class_prediction == test_label:", class_prediction == test_label)
#                 testLabel.append(test_label)
#                 predictaedLabel.append(current_seq)



#                 if(class_prediction == test_label): #(2,3 ) match 
#                   numCorrect = numCorrect+1
#               acc = numCorrect/numTest
#               #print("acc:", acc) 

#               #print("testLabel[]: ",testLabel)
#               #print("predictaedLabel[]: ",predictaedLabel)

#               fgdg=round(acc, 2)
#               #print("fgdg:", fgdg) 
#               if fgdg  > maxAcc:
#                 maxAcc=acc
#                 print(bcolors.WARNING + "maxAcc:" + bcolors.ENDC,maxAcc)
#                 max_classification_report_dict=metrics.classification_report(testLabel, predictaedLabel, digits=3,output_dict=True)

#         plt.plot(np.array(loss_values), 'r')
#         plt.xlabel('Epochs')
#         plt.ylabel('Total_Loss')
#         classification_report_dict.append(max_classification_report_dict)   
#         #print('classification_report_dict : \n',classification_report_dict)
#         Accuracy.append(maxAcc)  
#         #print('Accuracy : \n',Accuracy)
#     mean_acc = doClassSpecificCalulcation(Accuracy,trainLebel,classification_report_dict)
#     return mean_acc


In [ ]:
# def doClassSpecificCalulcation(Accuracy,trainLebel,classification_report_dict):
#   mean_acc = np.mean(Accuracy)
#   print('\np.mean(Accuracy) :',np.mean(Accuracy))
#   print('\np.std(Accuracy) :',np.std(Accuracy))
#   print('\n33333333 p.mean np.std(Accuracy) :     ',np.round(np.mean(Accuracy),2),"+-",np.round(np.std(Accuracy),2) )
#   for j in range( len(np.unique(trainLebel)) ):
#     print('\n\n\n\nclass :',j) 
#     precision=[]
#     recall=[]
#     f1_score=[]
#     for i in range(len(classification_report_dict)):
#       report=classification_report_dict[i]
#       #print('classification_report : \n',report) 
#       temp=report[str(j)]['precision'] 
#       precision.append(temp)

#       temp=report[str(j)]['recall'] 
#       recall.append(temp)

#       temp=report[str(j)]['f1-score'] 
#       f1_score.append(temp)

#     print('\np.mean(precision) \t p.mean(recall) \t p.mean(f1_score) :') 


#     print(np.mean(precision)) 
#     print(np.mean(recall)) 
#     print(np.mean(f1_score))

#     print('\np.mean p.std(precision) \tp.mean  p.std(recall) \tp.mean  p.std(f1_score) :')

#     print(np.round(np.mean(precision),2),"+-",np.round(np.std(precision),2) )
#     print(np.round(np.mean(recall),2),"+-",np.round(np.std(recall),2) )
#     print(np.round(np.mean(f1_score),2),"+-",np.round(np.std(f1_score),2) )
#   return mean_acc

In [ ]:
# def doWork2():

#   #test_sizes=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
#   test_sizes=[0.3]
#   for temp4 in range(len(test_sizes)):
#     test_size=test_sizes[temp4]
#     print("\n\n\n *************** test_size: ",test_size)
#     random_state=random.randint(42, 100)
#     print("random_state: ",random_state)
    
#     X_train, X_test, y_train, y_test = train_test_split(trainData, trainLebel, test_size=test_size, random_state=random_state)
#     #print("X_train.shape: ",X_train.shape)
#     #print("X_test.shape: ",X_test.shape)
#     #print("y_train.shape: ",y_train.shape)
#     #print("y_test.shape: ",y_test.shape)
#     #print(type(X_train))



#     print("X_train.shape X_test.shape y_train.shape y_test.shape ",
#               X_train.shape, X_test.shape ,y_train.shape, y_test.shape)
#     mean_acc = doWork( X_train, X_test, y_train, y_test)
#   return mean_acc

## **Run Main (doWork2)**

In [ ]:
# doWork2()

# Checking if results are consistent : **STD DEV across 20 runs**


In [ ]:
# list_mean_acc = []
# for _ in range(20):
#   mean_acc = doWork2()
#   list_mean_acc.append(mean_acc)